# KinSight
A family-focused neural network to identify my household people and paws.

This is am image classifier based on pre-trained CNN ResNet.

# 1. Dataset Format

family_dataset/
│
├── train/
│   ├── Marvin/
│   │   ├── img001.jpg
│   │   ├── img002.jpg
│   │   └── ...
│   ├── Jane/
│   ├── Eric/
│   ├── Tobin/
│   └── Wesley/
│
└── test/
    ├── Marvin/
    ├── Jane/
    ├── Eric/
    ├── Tobin/
    └── Wesley/

# Choosing this format is because:
- PyTorch’s ImageFolder class automatically reads class names from the subfolder names.
- It returns each image as a tensor and its class label as an integer.
- It avoids custom binary formats like IDX — your dataset stays human-readable and easy to extend.


In [9]:
# 2. Loading your dataset in PyTorch

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

train_dataset = datasets.ImageFolder(root='family_dataset/train', transform=transform)
test_dataset  = datasets.ImageFolder(root='family_dataset/test', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [10]:
# 3. Model choices
# We can start with a small pre-trained CNN, e.g.:
import torch
from torchvision import models

model = models.resnet18(weights='IMAGENET1K_V1')
model.fc = torch.nn.Linear(model.fc.in_features, 5)  # 5 classes: Marvin, Jane, Eric, Tobin, Wesley

This is called transfer learning — you’re reusing a model that already knows visual features (edges, shapes, textures) and just retraining the final layer for your own categories.

In [11]:
# 4. Training loop

import torch.nn as nn
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(7):
    
    model.train()

    for imgs, labels in train_loader:
        
        optimizer.zero_grad()
    
        outputs = model(imgs)
    
        loss = criterion(outputs, labels)
    
        loss.backward()
    
        optimizer.step()
    
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")


Epoch 1, Loss: 0.9322
Epoch 2, Loss: 0.7035
Epoch 3, Loss: 1.2983
Epoch 4, Loss: 0.3093
Epoch 5, Loss: 0.0741
Epoch 6, Loss: 0.0777
Epoch 7, Loss: 0.0462


In [13]:
# 5. Evaluation

correct, total = 0, 0

model.eval()

with torch.no_grad():
    for imgs, labels in test_loader:
        outputs = model(imgs)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

print(f"Accuracy: {100 * correct / total:.2f}%")

Accuracy: 37.50%
